In [2]:
import json
import sys
import os
import shutil


import requests
import yaml
from PIL import Image

disease_list = ['출혈', '결손', '괴사']
dir_num = str('V')

In [67]:
origin_json = 'C:/Users/user/fish_disease_dataset/original/어류개체촬영영상_질병어류 바운딩박스/dtset' + dir_num + '_disease.json' # 원본 json 파일 경로
save_dir = 'C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/labels/dtset' + dir_num + '_disease/' # 라벨 파일 저장 디렉터리

# 이미지 번호로 이미지 파일 정보를 찾아서 반환
# images: 이미지 정보 딕셔너리
# image_id: 검색할 이미지 번호
# info_type: 찾을 정보 종류
def find_image_info(images, image_id, info_type):
    info = ''
    for i in images:
        if i['id'] == image_id:
            if info_type == 'file_name':
                info = i['file_name']
            elif info_type == 'width':
                info = i['width']
            elif info_type == 'height':
                info = i['height']
    return info

# bbox 값을 yolo 형식으로 변환
# bbox: bbox 값
# size: 이미지 사이즈
# json bbox 속성: x, y, width, height (x, y는 바운딩 박스의 왼쪽 위 꼭짓점 값)
def calc_yolo_bbox(bbox, size):
    
    dw = 1./size[0] 
    dh = 1./size[1] 

    w = bbox[2]
    h = bbox[3]
    x_center = (w / 2) + bbox[0]
    y_center = (h / 2) + bbox[1]
       
    x_center = round(x_center * dw, 6)
    y_center = round(y_center * dh, 6) 
    w = round(w * dw, 6) 
    h = round(h * dh, 6) 

    return (x_center,y_center,w,h) 

# 라벨을 txt 파일로 저장
# classes: 클래스 값(질병)
# bbox: 질병 bbox 값
# file_name: 저장 파일 이름
def save_label_to_txt(classes, bbox, file_name):
    contents = [classes, bbox[0], bbox[1], bbox[2], bbox[3]]
    content = ' '.join(map(str,contents))
    content = content + '\n'
    with open(save_dir + file_name[:-4] + '.txt', 'w') as f:
        f.write(content)
        
# 어류 바운딩 박스를 라벨에 추가
# bbox: 어류 bbox 값
# file_name: 저장 파일 이름        
def add_fish_to_label(bbox, file_name):
    contents = [len(disease_list), bbox[0], bbox[1], bbox[2], bbox[3]]
    content = ' '.join(map(str,contents))
    with open(save_dir + file_name[:-4] + '.txt', 'a') as f:
        f.write(content)

# 변환 함수(Convert JSON labels to YOLO labels)
# file: json 파일 경로
def convert(file):
    classes = 0 # 클래스 번호(classes.txt)
    
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)  # load JSON
    
    images = data['images'] # 이미지 정보 딕셔너리
    ants = data['annotations'] # 어노테이션 정보 딕셔너리
    
    for i in ants:
        if i['diseases_exist'] is True: # 질병이 있는 물고기
            if i['category_id'] == 5:  # 돌돔
                file_name = find_image_info(images, i['image_id'], 'file_name')
                temp = i['diseases_desc'].split(" ")[1]
                desc = temp[0:2]
                for d in disease_list:
                    if d == desc: # disease_list에 등록된 질병
                        classes = disease_list.index(d)
                        img_size = [find_image_info(images, i['image_id'], 'width'), 
                                    find_image_info(images, i['image_id'], 'height')]

                        yolo_bbox = calc_yolo_bbox(i['diseases_bbox'], img_size) # 라벨 형식 변환
                        fish_bbox = calc_yolo_bbox(i['bbox'], img_size)
        
                        save_label_to_txt(classes, yolo_bbox, file_name) 
                        add_fish_to_label(fish_bbox, file_name)
    
    
#     for img in tqdm(data, desc=f'Converting {file}'):\
#         im_path = img['Labeled Data']
#         im = Image.open(requests.get(im_path, stream=True).raw if im_path.startswith('http') else im_path)  # open
#         width, height = im.size  # image size
#         label_path = save_dir / 'labels' / Path(img['External ID']).with_suffix('.txt').name
#         image_path = save_dir / 'images' / img['External ID']
#         im.save(image_path, quality=95, subsampling=0)

#         for label in img['Label']['objects']:
#             # box\
#             top, left, h, w = label['bbox'].values()  # top, left, height, width
#             xywh = [(left + w / 2) / width, (top + h / 2) / height, w / width, h / height]  # xywh normalized

#             # class
#             cls = label['value']  # class name
#             if cls not in names:
#                 names.append(cls)

#             line = names.index(cls), *xywh  # YOLO format (class_index, xywh)
#             with open(label_path, 'a') as f:
#                 f.write(('%g ' * len(line)).rstrip() % line + '\n')

#     # Save dataset.yaml
#     d = {'path': f"../datasets/{file.stem}  # dataset root dir",
#          'train': "images/train  # train images (relative to path) 128 images",
#          'val': "images/val  # val images (relative to path) 128 images",
#          'test': " # test images (optional)",
#          'nc': len(names),
#          'names': names}  # dictionary

#     with open(save_dir / file.with_suffix('.yaml').name, 'w') as f:
#         yaml.dump(d, f, sort_keys=False)

#     print('Conversion completed successfully!')

# 메인 함수: 위 변환 함수 호출하기
if __name__ == '__main__':
    convert(origin_json)

In [69]:
# 추출한 라벨에 해당하는 이미지 복사해오기
source_img_dir = "C:/Users/user/fish_disease_dataset/original/어류개체촬영영상_질병어류 바운딩박스/images/dtset" + dir_num + "_disease/"
image_list = os.listdir(source_img_dir)

label_dir = "C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/labels/dtset" + dir_num + "_disease/"
label_list = os.listdir(label_dir)

target_dir = "C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/images/dtset" + dir_num + "_disease/"

for image in image_list:
    for label in label_list:
        if image[:-4] == label[:-4]:
            shutil.copy(source_img_dir + image, target_dir + image)

In [57]:
# 특정 질병을 가진 이미지 파일 이름 출력
target_dir = "C:/Users/user/fish_disease_dataset/multy_class/set/originwithaug/labels/"
label_list = os.listdir(target_dir)
find_disease = ""
for label in label_list:
    if label == "classes.txt":
        continue
    with open (target_dir + label, "r") as f:
        disease = int(f.read().split(" ")[0])
    if disease == disease_list.index(find_disease):
        print(label)

ValueError: '' is not in list

In [6]:
# 주어진 라벨 폴더에서 각 질병의 개수 파악하기
# target_dir = "C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/labels/dtset" + "V" + "_disease/"
target_dir = "C:/Users/user/fish_disease_dataset/multy_class_with_fish_bbox/set/set2/train/labels/"
label_list = os.listdir(target_dir)
disease_list = ["출혈", "결손", "괴사"]
disease_count = [0, 0, 0]
for label in label_list:
    if label == "classes.txt":
        continue
    with open (target_dir + label, "r") as f:
        disease = int(f.read().split(" ")[0])
        
#         print(label, " -> 질병: ", disease_list[disease])
        disease_count[disease] += 1

for i in range(len(disease_list)):
    print(disease_list[i], end=": ")
    print(disease_count[i])

출혈: 328
결손: 376
괴사: 392


In [5]:
# 정상 돌돔 이미지를 모두 복사
dir_num = str(1)
file = 'C:/Users/user/fish_disease_dataset/original/어류개체촬영영상/dtset' + dir_num + '_disease.json'

with open(file, 'r', encoding='utf-8') as f:
    data = json.load(f)  # load JSON
    
images = data['images'] # 이미지 정보 딕셔너리
ants = data['annotations'] # 어노테이션 정보 딕셔너리
    
for i in ants:
    if i['category_id'] == 5:  # 돌돔
        if i['diseases_exist'] is False: # 질병이 없는 물고기
            # 이미지 이름 출력
            file_name = find_image_info(images, i['image_id'], 'file_name')
            print(file_name)